In [ ]:
!mkdir data

In [ ]:
!wget https://zenodo.org/record/4277311/files/quotes-2020.json.bz2?download=1 -O data/quotebank2020.json.bz2

--2021-11-09 10:56:13--  https://zenodo.org/record/4277311/files/quotes-2020.json.bz2?download=1
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 830742956 (792M) [application/octet-stream]
Saving to: ‘data/quotebank2020.json.bz2’

data/quotebank2020. 100%[===================>] 792.26M  21.1MB/s    in 56s     

2021-11-09 10:57:15 (14.1 MB/s) - ‘data/quotebank2020.json.bz2’ saved [830742956/830742956]



In [12]:
!pip install pandas==1.0.5
!pip show pandas

Name: pandas
Version: 1.0.5
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: https://pandas.pydata.org
Author: None
Author-email: None
License: BSD
Location: /usr/local/lib/python3.7/dist-packages
Requires: numpy, pytz, python-dateutil
Required-by: xarray, vega-datasets, statsmodels, sklearn-pandas, seaborn, pymc3, plotnine, pandas-profiling, pandas-gbq, pandas-datareader, mlxtend, mizani, holoviews, gspread-dataframe, google-colab, fix-yahoo-finance, fbprophet, fastai, cufflinks, cmdstanpy, arviz, altair


In [3]:
!pip show pandas

Name: pandas
Version: 1.0.5
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: https://pandas.pydata.org
Author: None
Author-email: None
License: BSD
Location: /usr/local/lib/python3.7/dist-packages
Requires: pytz, numpy, python-dateutil
Required-by: xarray, vega-datasets, statsmodels, sklearn-pandas, seaborn, pymc3, plotnine, pandas-profiling, pandas-gbq, pandas-datareader, mlxtend, mizani, holoviews, gspread-dataframe, google-colab, fix-yahoo-finance, fbprophet, fastai, cufflinks, cmdstanpy, arviz, altair


In [18]:
import nltk

nltk.download('stopwords')
nltk.download('punkt')

from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

def prepare_tokens(tokens, stemmer):
  return map(
      # to avoid any multithreading issues, stemmer is always passed from outside
      stemmer.stem, 
      map(lambda word: word.lower(), tokens)
  )

def is_topic_related(text, term_set_to_look_for, stemmer):
  tokens = prepare_tokens(word_tokenize(text), stemmer)
  return bool(set(tokens) & term_set_to_look_for)

class ChunkProcessingFilter:
  def __init__(self, *terms_to_look_for):
    self.set_to_look_for = set(
        prepare_tokens(terms_to_look_for, PorterStemmer()))

  def __call__(self, chunk):
    stemmer = PorterStemmer()
    mask = chunk.apply(
        lambda row: is_topic_related(
            row.quotation, self.set_to_look_for, stemmer),
        axis=1)
    return chunk[mask]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [19]:
import pandas as pd
import itertools
from functools import reduce
from multiprocessing import Pool

df_reader = pd.read_json('data/quotebank2020.json.bz2', lines=True, compression='bz2', error_bad_lines=False, chunksize=200000)
try:
  processing_filter = ChunkProcessingFilter('vegan', 'vegetarian', 'veget', 'plant-based', 'meat')
  with Pool(2) as pool:
    filtered_entries = reduce(
        lambda acc, new_el: pd.concat([acc, new_el]),
        pool.imap_unordered(processing_filter, df_reader)
    )
finally:
  df_reader.close()

filtered_entries.to_json('data/filtered2020.json.bz2', compression='bz2', orient='records', lines=True)

Process ForkPoolWorker-8:
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "<ipython-input-18-1d99f86eaad5>", line 30, in __call__
    axis=1)
  File "/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py", line 6878, in apply
    return op.get_result()
Process ForkPoolWorker-7:
  File "/usr/local/lib/python3.7/dist-packages/pandas/core/apply.py", line 186, in get_result
    return self.apply_standard()
  File "/usr/local/lib/python3.7/dist-packages/pandas/core/apply.py", line 296, in apply_standard
    values, self.f, axis=self.axis, dummy=dummy, labels=labels
Traceback (most recent call last):
  File "pandas/_libs/reduction.pyx", line 620, in pandas._libs.reductio

KeyboardInterrupt: ignored

In [17]:
filtered_entries

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
1967,2020-02-13-100687,This diet focuses on lifelong changes to healt...,None,[],2020-02-13 18:22:19,1,"[[None, 0.8001], [Dr. Sebi, 0.1999]]",[https://parade.com/995529/christinperry/dr-se...,E
2692,2020-02-07-007719,"As an empowered female leader myself, I am thr...",None,[],2020-02-07 13:09:21,1,"[[None, 0.9507], [Maye Musk, 0.0493]]",[https://www.perishablenews.com/produce/celebr...,E
4263,2020-02-13-087416,the funniest thing they left behind was a big ...,None,[],2020-02-13 17:00:30,1,"[[None, 0.8424], [Chevy Chase, 0.1576]]",[https://www.washingtonian.com/2020/02/13/supe...,E
4813,2020-01-21-098307,We stack the freshly plucked fruits into three...,Sahi Ram,[Q19605026],2020-01-21 12:49:00,1,"[[Sahi Ram, 0.9035], [None, 0.0965]]",[http://freshplaza.com/article/9181949/the-tin...,E
5263,2020-02-12-011869,Burger Burger will use local produce as much a...,None,[],2020-02-12 18:56:09,1,"[[None, 0.9078], [El Chapo, 0.0922]]",[https://www.belfastlive.co.uk/whats-on/food-d...,E
...,...,...,...,...,...,...,...,...,...
46775,2020-02-07-038035,I think you can still go out and get groceries...,Cameron Walker,"[Q25189341, Q30122762]",2020-02-07 23:01:02,1,"[[Cameron Walker, 0.9119], [None, 0.0881]]",[http://www.theglobeandmail.com/world/article-...,E
46886,2020-02-11-041875,"If you look at meat processing, if you look at...",Sarah Miller,"[Q23881549, Q57435876]",2020-02-11 10:00:19,1,"[[Sarah Miller, 0.9027], [None, 0.0973]]",[http://mobile.nytimes.com/2020/02/11/technolo...,E
48087,2020-01-18-047886,Veganuary exists to help people discover delic...,Wendy Matthews,[Q15524937],2020-01-18 10:00:09,3,"[[Wendy Matthews, 0.7805], [None, 0.195], [Jos...",[http://www.nytimes.com/2020/01/18/style/self-...,E
48538,2020-01-24-001965,A phrase that Ash says the whole time after cr...,Jonny Allan,[Q6275710],2020-01-24 16:06:00,1,"[[Jonny Allan, 0.806], [None, 0.194]]",[http://www.independent.co.uk/arts-entertainme...,E


In [32]:
filtered_entries.to_json('data/filtered2020.json.bz2', compression='bz2', orient='records', lines=True)